# Data Story 

Link to Data: https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-02-11/hotels.csv

## Project Proposal

My project will seek to solve a common but surging issue for the hotel industry, predicting cancellations and maintaining brand integrity among surging customer interest in booking through online travel agents, rather than direct. According to a study conducted by D-Edge Hospitality Solutions, cancellation rates in the hotel industry increased more than 8 percent from 2014 to 2017^[Hertzfeld, Esther. Study: Cancellation Rate at 40% as OTAs Push Free Change Policy. Hotel Management, 23 Apr. 2019, www.hotelmanagement.net/tech/study-cancelation-rate-at-40-as-otas-push-free-change-policy.
]

The impact on the industry has come from pressure primarily stemming from online travel agencies and their adoption of ‘Risk Free Reservations’. While OTA’s use cancellations as a way to expand their market availability and retain customer loyalty, hotels risk the ability to forecast revenue and maintain brand integrity in the process^[Funnell, Rob. “The Real Cost of 'Free' Cancellations for Hotels.” Triptease, Triptease - Attract. Convert. Compete., 13 May 2019,
www.triptease.com/blog/the-real-cost-of-free-cancellations/?utm_source=MediaPartner&utm_medium=HotelSpeak.
]

Hotels often desire to create a personalized ease of  service for customers from the moment of booking, but OTA’s want their customers to adopt a ‘book now, ask questions later’ mentality. The preference for customers often falls towards the OTA’s, because they will often advertise a lower price than booking direct, as they absorb the cancellation risk, which creates a strain on the hotels’ customer relationship and diminishes the booking experience. While OTA’s do draw in customers and expand outreach for hotels, there is an opportunity to optimize customer channels and lower risk, while increasing real-time income.

The outcome of this project will give the client insights into predicting cancellations and provide suggestions for modifying their cancellation policy. This will provide the hotel with the ability to optimize customer channels, maintain brand integrity and increase customer loyalty.

The dataset was obtained from [tidy tuesday](https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-02-11/hotels.csv) but originated from [Science Direct](https://www.sciencedirect.com) and contains a collection of observations taken from 2015 to 2017 of guest bookings for two hotels, both located in Portugal. The data was collected directly from the hotels’ PMS (property management system) database and was relatively clean and structured upon retrieval. Each observation represents a booking, with the variables capturing details including when it was booked, dates of stay, through which operator the guest chose to go through, if that booking was cancelled and the average daily rate. Once this project is complete, I will be able to provide insights on the following questions: 


> Can we predict if a guest will cancel a reservation at the moment of booking? What inferences can we draw from cancellation patterns that would help optimize customer channels and lower the overall risk for a hotel?

I will attempt to solve this problem by investigating where cancellations primarily occur and during what time of the year. After drawing insights from the exploratory data analysis phase, the dataset will be modified for the modeling process with the goal of predicting the cancelation column of the set. Insights from the model and from the exploratory phase will lead to suggestions to the client that will help them forecast cancellations and optimize customer channels as mentioned above. 

The client will receive the working model in the form of a web application as well as a summary paper and slide deck. The slide deck will summarize the findings and attempt to “sell” the work to the client, and the paper will summarize the details. Code will be provided for inspection and reproducibility for the client. 

## Data Wrangling and Cleaning steps

The data I obtained was relatively clean to begin with, however there were a few instances of “NULL” and empty entries as well as mislabeled entries that needed clarification. The convention for null data was a string with 7 spaces before NULL ("NULL"), so this was found and replaced with `numpy`’s NAN entry for ease of identification. At this point it was a matter of understanding the data to replace NAN values with appropriate labels for the variable. 

The variables with NAN entries were the company, agent and country columns. Upon more exploration of the data and its source website, it was discovered that for company and agent, null entries correlated to customers that did not go through a company or agent to book. To correct this, the original company and agent names were replaced with string numerics to protect anonymity so to continue with this convention, nan entries were replaced with “0” to represent “no agent” or “no company”. The country column had no added clarification for the nan entries, so it was decided that “UNK” for unknown would be used instead. 
The last step for cleaning was to create a datetime column for a guest's date of arrival. The data provided the day, month and year separately for arrival, so these were combined and converted to datetime format to allow more versatility in analysis. 

Lastly, the data was explored to identify any outliers that may skew analysis. A few peculiar entries were found. For example, one group booked 2 years in advance and one brought 10 children and 10 kids, one guest also had 26 previous cancellations. Upon further exploration there was no clear evidence to rule out these observations, so no steps were taken to alter these data. 

### Import appropriate libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import os

### Read in raw data

In [2]:
#Read in the data
hotels = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-02-11/hotels.csv')

In [3]:
hotels.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


### Data Cleaning

In [4]:
# convert 'NULL' entries to np.nan
cols = ['agent', 'company']
for col in cols:
    for i in range(len(hotels[col])):
        if (hotels[col].iloc[i] == '       NULL'):
            hotels[col].iloc[i] = np.nan
        else:
            continue

In [5]:
# Insert 0 for each missing Children entry and convert to integer.
hotels.children = pd.to_numeric(hotels.children.replace(np.nan, 0), downcast = 'integer')

In [6]:
# Convert missing values for Company and Agent to 0, meaning 'no agent' and 'no company'.
cols = ['company', 'agent']
for col in cols:
        hotels[col] = hotels[col].replace(np.nan, '0')

In [7]:
# If country of origin is missing, replace as 'UNK' for unknown.
hotels['country'] = hotels['country'].replace(np.nan, 'UNK')

In [8]:
# convert reservation_status_date to datetime
hotels['reservation_status_date'] = pd.to_datetime(hotels['reservation_status_date'])

In [9]:
# drop undefined values
hotels = hotels[hotels.market_segment != 'Undefined']

In [10]:
# drop babies outlier
hotels.drop(hotels[hotels.babies >=5].index, inplace=True)

In [11]:
# replace any negative values for adr with zero using clip
hotels['adr'] = hotels.adr.clip(lower=0)

In [12]:
# Generalize children and babies to "kids"
hotels['kids'] = hotels.children + hotels.babies

hotels = (hotels.assign(kids = lambda df: df.kids.map(
                    lambda kids: 'has_kids' if kids > 0 else 'no_kids')))

In [13]:
hotels.name = 'Hotels'
null_data = hotels[hotels.isnull().any(axis=1)]
if null_data.empty:
    print(hotels.name + ' contains no null values')
else:
    print(hotels.name + ' does contain null values')

Hotels contains no null values


In [14]:
hotels.to_csv(r'/Users/mattmerrill/Springboard/Capstone/Predicting_Hotel_Cancellations/Jupyter_Notebooks/data/hotels_clean.csv', index=True)

In [49]:
ms_lead = hotels[['is_canceled', 'market_segment', 'lead_time', 'arrival_date_year']]
ms_lead

,is_canceled,market_segment,lead_time,arrival_date_year
0,0,Direct,342,2015
1,0,Direct,737,2015
2,0,Direct,7,2015
3,0,Corporate,13,2015
4,0,Online TA,14,2015
...,...,...,...,...
119385,0,Offline TA/TO,23,2017
119386,0,Online TA,102,2017
119387,0,Online TA,34,2017
119388,0,Online TA,109,2017


In [64]:
ms_lead[(ms_lead.lead_time >= 250) & \
        (ms_lead.market_segment != 'Online TA') & \
        (ms_lead.is_canceled == 1) & \
        (ms_lead.arrival_date_year == 2016)]

,is_canceled,market_segment,lead_time,arrival_date_year
3819,1,Offline TA/TO,284,2016
3960,1,Offline TA/TO,312,2016
4948,1,Groups,275,2016
4956,1,Groups,275,2016
4957,1,Groups,275,2016
...,...,...,...,...
82752,1,Offline TA/TO,325,2016
82753,1,Offline TA/TO,325,2016
85211,1,Corporate,304,2016
87687,1,Direct,315,2016


In [66]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119386 entries, 0 to 119389
Data columns (total 33 columns):
hotel                             119386 non-null object
is_canceled                       119386 non-null int64
lead_time                         119386 non-null int64
arrival_date_year                 119386 non-null int64
arrival_date_month                119386 non-null object
arrival_date_week_number          119386 non-null int64
arrival_date_day_of_month         119386 non-null int64
stays_in_weekend_nights           119386 non-null int64
stays_in_week_nights              119386 non-null int64
adults                            119386 non-null int64
children                          119386 non-null int8
babies                            119386 non-null int64
meal                              119386 non-null object
country                           119386 non-null object
market_segment                    119386 non-null object
distribution_channel              119386 non-

bookings from ota's with lead times greater than 250 days had a cancellation rate of 64% in 2017
- The other methods of booking in 2017 had a cancellation rate of 47%
bookings from ota's with lead time less than 10 days had a cancellation rate of 12%

In [46]:
hotels.distribution_channel.unique()

array(['Direct', 'Corporate', 'TA/TO', 'Undefined', 'GDS'], dtype=object)